In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda')

In [3]:
train_png = sorted(glob('/mnt/aai/shin/dacon/data/train/*.png'))
test_png = sorted(glob('/mnt/aai/shin/dacon/data/val/*.png'))

In [4]:
train_y = pd.read_csv("/mnt/aai/shin/dacon/data/train_df_1.csv")
test_y = pd.read_csv("/mnt/aai/shin/dacon/data/train_df_2.csv")

train_labels = train_y["class"]
test_labels = test_y["class"]


label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]
test_labels = [label_unique[k] for k in test_labels]


In [5]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (128, 128))
    return img

In [6]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 477/477 [00:24<00:00, 19.29it/s]


In [7]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=15)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [8]:
batch_size = 32
epochs = 8

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(test_labels), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [14]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

def acc_function(real, pred):
    score = accuracy_score(real, pred)
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 


best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    train_acc = acc_function(train_y, train_pred)
    train_f1 = score_function(train_y, train_pred)
    
    # valid
    test_pred=[]
    test_y=[]
    model.eval()
    f_pred = []

    with torch.no_grad():
        for batch in (test_loader):
            x = torch.tensor(batch[0], dtype = torch.float32, device = device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())
            test_y += y.detach().cpu().numpy().tolist()

    val_acc = acc_function(test_y, f_pred)    
    
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}     accuracy : {train_acc:.5f}')
    print(f'VALID    accuracy : {val_acc:.5f}')

[8, 6, 3, 3, 7, 1, 14, 10, 1, 5, 9, 6, 6, 3, 12, 9, 1, 6, 1, 2, 8, 1, 1, 4, 14, 5, 4, 14, 1, 3, 7, 7, 1, 5, 0, 2, 7, 8, 2, 14, 6, 4, 5, 2, 9, 1, 9, 8, 6, 2, 2, 11, 7, 10, 8, 9, 1, 10, 4, 9, 5, 7, 5, 14, 13, 5, 10, 9, 7, 2, 8, 13, 0, 5, 0, 13, 5, 13, 14, 8, 10, 6, 13, 8, 14, 6, 4, 4, 9, 3, 5, 9, 10, 0, 3, 4, 12, 6, 10, 13, 6, 4, 9, 3, 6, 8, 5, 2, 3, 10, 8, 4, 5, 2, 11, 4, 6, 8, 5, 9, 11, 1, 13, 14, 7, 8, 4, 3, 4, 13, 6, 10, 4, 11, 3, 1, 10, 7, 7, 5, 3, 13, 8, 13, 10, 8, 3, 12, 4, 13, 3, 2, 8, 10, 6, 7, 5, 9, 4, 5, 2, 3, 12, 9, 1, 9, 10, 1, 1, 0, 13, 10, 12, 10, 3, 2, 0, 6, 6, 12, 9, 0, 4, 5, 7, 7, 7, 8, 10, 4, 4, 14, 3, 4, 9, 14, 13, 5, 2, 6, 10, 6, 0, 2, 2, 7, 2, 3, 5, 9, 6, 14, 12, 3, 12, 8, 5, 6, 5, 2, 10, 3, 9, 4, 6, 5, 5, 10, 10, 5, 3, 0, 1, 6, 12, 2, 7, 9, 10, 10, 1, 2, 0, 2, 1, 4, 14, 4, 2, 13, 2, 6, 5, 11, 9, 8, 5, 9, 3, 3, 2, 14, 11, 1, 14, 10, 9, 7, 9, 13, 14, 9, 6, 2, 9, 3, 7, 9, 5, 9, 9, 14, 9, 5, 4, 14, 5, 9, 7, 0, 12, 9, 7, 10, 1, 9, 5, 5, 4, 10, 0, 8, 13, 7, 1, 2, 12, 7, 

In [16]:
label_decoder = {val:key for key, val in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

test_y = pd.read_csv("/mnt/aai/shin/dacon/data/train_df_2.csv")
test_y["predict"] = f_result
test_y

,index,file_name,class,state,label,predict
0,3800,13800.png,pill,good,pill-good,pill
1,3801,13801.png,leather,good,leather-good,leather
2,3802,13802.png,carpet,good,carpet-good,carpet
3,3803,13803.png,carpet,good,carpet-good,carpet
4,3804,13804.png,metal_nut,flip,metal_nut-flip,metal_nut
...,...,...,...,...,...,...
472,4272,14272.png,transistor,good,transistor-good,transistor
473,4273,14273.png,transistor,good,transistor-good,transistor
474,4274,14274.png,grid,good,grid-good,grid
475,4275,14275.png,zipper,good,zipper-good,zipper
